<a href="https://colab.research.google.com/github/zqiaohe/100daysofcode/blob/master/CupIT2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.read_csv("/content/drive/My Drive/CupIT2020/train_data.csv",index_col=[1], parse_dates=[1] ) 
#df = pd.read_csv("/content/drive/My Drive/CupIT2020/train_data.csv") 

In [0]:
minidf = df[(df['cif_id']==335992)|(df['cif_id']==234815)|(df['cif_id']==198842)].sort_index(ascending=False)

In [0]:
minidf

,cif_id,gi_smooth_3m,big_city,cu_gender,cu_education_level,cu_empl_area,cu_empl_level,payroll_f,cur_quantity_pl,cur_quantity_mort,cur_quantity_cc,cur_quantity_deposits,cur_quantity_dc,cur_quantity_accounts,cur_quantity_saccounts,cur_quantity_mf,cc_balance,cl_balance,ml_balance,pl_balance,td_volume,ca_volume,sa_volume,mf_volume,dc_cash_spend_v,dc_cash_spend_c,cc_cash_spend_v,cc_cash_spend_c,dc_pos_spend_v,dc_pos_spend_c,cc_pos_spend_v,cc_pos_spend_c,ca_f,rc_session_qnt_cur_mon,cur_qnt_sms,active,standalone_dc_f,standalone_payroll_dc_f,standalone_nonpayroll_dc_f,salary,cu_age,cu_mob,cu_empl_cur_dur_m,is_married,cu_eduaction_level
dlk_cob_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-12-31,234815,7.303600,MSK,1,NaN,NaN,SUPPORT STAFF,1,0,0,0,0,1,1,0,0,NaN,NaN,NaN,NaN,NaN,311.22072,NaN,NaN,600.0,2,0.0,0,474.79416,8,0.0,0,1,0,-100.0,1,1,1,0,1360.09368,46.0,161,187.0,1,-100
2018-12-31,335992,10.812928,MLN,0,02. Среднее профессиональное,ТОРГОВЛЯ,SUPPORT STAFF,0,1,0,0,0,1,1,0,0,NaN,NaN,NaN,-1027.11120,NaN,8.60448,NaN,NaN,0.0,0,0.0,0,0.00000,0,0.0,0,1,0,-100.0,1,0,0,0,1192.80000,30.0,150,52.0,1,2
2018-12-31,198842,4.287918,SPB,0,NaN,NaN,NaN,1,0,0,0,0,1,1,0,0,NaN,NaN,NaN,NaN,NaN,206.83536,NaN,NaN,2424.0,5,0.0,0,145.41024,8,0.0,0,1,0,1.0,1,0,0,0,1801.29600,45.0,23,83.0,0,-100
2018-11-30,234815,7.346214,MSK,1,NaN,NaN,SUPPORT STAFF,1,0,0,0,0,1,1,0,0,NaN,NaN,NaN,NaN,NaN,25.92120,NaN,NaN,828.0,3,0.0,0,557.39112,10,0.0,0,1,0,-100.0,1,1,1,0,1456.58400,46.0,160,186.0,1,-100
2018-11-30,335992,11.729811,MLN,0,02. Среднее профессиональное,ТОРГОВЛЯ,SUPPORT STAFF,0,1,0,0,0,1,1,0,0,NaN,NaN,NaN,-1143.90552,NaN,8.94072,NaN,NaN,0.0,0,0.0,0,0.00000,0,0.0,0,1,0,-100.0,1,0,0,0,1192.80000,30.0,149,51.0,1,2
2018-11-30,198842,10.635166,SPB,0,NaN,NaN,NaN,1,0,0,0,0,1,1,0,0,NaN,NaN,NaN,NaN,NaN,13.19760,NaN,NaN,1800.0,2,0.0,0,0.00000,0,0.0,0,1,0,1.0,1,0,0,0,1775.88000,45.0,22,82.0,0,-100
2018-10-31,198842,5.148496,SPB,0,NaN,NaN,NaN,1,0,0,0,0,1,1,0,0,NaN,NaN,NaN,NaN,NaN,13.34160,NaN,NaN,2376.0,2,0.0,0,0.00000,0,0.0,0,1,0,1.0,1,0,0,0,1166.25600,45.0,21,81.0,0,-100
2018-10-31,335992,12.874664,MLN,0,02. Среднее профессиональное,ТОРГОВЛЯ,SUPPORT STAFF,0,1,0,0,0,1,1,0,0,NaN,NaN,NaN,-1259.37000,NaN,9.27696,NaN,NaN,0.0,0,0.0,0,0.00000,0,0.0,0,1,0,-100.0,1,0,0,0,1192.80000,30.0,148,50.0,1,2
2018-10-31,234815,21.700902,MSK,1,NaN,NaN,SUPPORT STAFF,1,0,0,0,0,1,1,0,0,NaN,NaN,NaN,NaN,NaN,352.36032,NaN,NaN,1920.0,4,0.0,0,711.30720,20,0.0,0,1,0,-100.0,1,1,1,0,1456.58400,46.0,159,185.0,1,-100


In [0]:
minidf.groupby('cif_id').rolling(window=6) .sum()['gi_smooth_3m']

cif_id  dlk_cob_date
198842  2018-12-31             NaN
        2018-11-30             NaN
        2018-10-31             NaN
        2018-09-30             NaN
        2018-08-31             NaN
        2018-07-31       55.383146
        2018-06-30       63.243903
        2018-05-31       58.439354
        2018-04-30       56.630070
        2018-03-31       49.935738
        2018-02-28       42.639613
        2018-01-31       28.908961
234815  2018-12-31             NaN
        2018-11-30             NaN
        2018-10-31             NaN
        2018-09-30             NaN
        2018-08-31             NaN
        2018-07-31      138.529499
        2018-06-30      158.372263
        2018-05-31      175.660444
        2018-04-30      176.873340
        2018-03-31      166.575638
        2018-02-28      147.225976
        2018-01-31      135.085898
335992  2018-12-31             NaN
        2018-11-30             NaN
        2018-10-31             NaN
        2018-09-30             NaN

In [0]:
minidf.groupby('cif_id').rolling(window=2) .sum()['gi_smooth_3m']

cif_id  dlk_cob_date
198842  2018-12-31            NaN
        2018-11-30      14.923085
        2018-10-31      15.783662
        2018-09-30      14.475552
        2018-08-31      18.803378
        2018-07-31      25.984510
        2018-06-30      28.656863
        2018-05-31      17.979292
        2018-04-30       9.169829
        2018-03-31       5.971937
        2018-02-28       4.812921
        2018-01-31       4.957732
234815  2018-12-31            NaN
        2018-11-30      14.649814
        2018-10-31      29.047116
        2018-09-30      52.790070
        2018-08-31      70.796663
        2018-07-31      71.089616
        2018-06-30      58.528484
        2018-05-31      51.780758
        2018-04-30      47.548193
        2018-03-31      43.705263
        2018-02-28      41.149299
        2018-01-31      39.599877
335992  2018-12-31            NaN
        2018-11-30      22.542739
        2018-10-31      24.604475
        2018-09-30      26.734614
        2018-08-31      28.

In [0]:
new_train = df.sort_index(ascending=False)
#new_train.groupby('cif_id').apply(lambda x: x.sort_index(ascending=False)).rolling(window=6).sum()['gi_smooth_3m']

In [0]:
new_train = new_train.groupby('cif_id').rolling(window=2) .sum()['gi_smooth_3m']

In [162]:
new_train.head()

cif_id  dlk_cob_date
1001    2018-12-31             NaN
        2018-11-30       21.908271
        2018-10-31      144.683255
        2018-09-30      220.462897
        2018-08-31      183.229231
Name: gi_smooth_3m, dtype: float64

In [172]:
new_train.keys

<bound method Series.keys of cif_id  dlk_cob_date
1001    2018-12-31             NaN
        2018-11-30       21.908271
        2018-10-31      144.683255
        2018-09-30      220.462897
        2018-08-31      183.229231
                           ...    
511537  2018-05-31       85.800848
        2018-04-30       85.386530
        2018-03-31       85.377042
        2018-02-28       87.307184
        2018-01-31      114.659481
Name: gi_smooth_3m, Length: 4161783, dtype: float64>

In [0]:
_train = pd.merge(df, new_train, on='key')

In [0]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [0]:
y_true = np.array([3, -0.5, 2, 7])
y_pred = np.array([2.5, -0.3, 2, 8])

In [0]:
mean_absolute_percentage_error(y_true, y_pred)

17.738095238095237

In [0]:
test_df = pd.read_csv("/content/drive/My Drive/CupIT2020/test_data.csv") 